In [41]:
#import dependencies
import pandas as pd
from matplotlib import pyplot as plt
#import numpy as np
import requests
import datetime
from scipy.stats import linregress

#known issues
## zipcode "Texas" to 75042
## 7038 zip needs to be 75038

In [42]:
#file path
file = "combinedmlsdata.csv"

#read the file into pandas
df = pd.read_csv(file, sep=',', error_bad_lines=False, index_col=False, dtype='unicode')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170557 entries, 0 to 170556
Data columns (total 32 columns):
 #   Column                           Non-Null Count   Dtype 
---  ------                           --------------   ----- 
 0   MLS #                            170557 non-null  object
 1   Unnamed: 1                       170557 non-null  object
 2   Address                          170557 non-null  object
 3   City                             170557 non-null  object
 4   Zip Code                         170557 non-null  object
 5   List Price                       170557 non-null  object
 6   Original List Price              170540 non-null  object
 7   SqFt Total                       170548 non-null  object
 8   Close Price                      133245 non-null  object
 9   RATIO Current Price By SQFT      170498 non-null  object
 10  Number Of Stories                170556 non-null  object
 11  Beds Total                       170557 non-null  object
 12  Baths Total     

In [43]:
df.columns.values.tolist()

['MLS #',
 'Unnamed: 1',
 'Address',
 'City',
 'Zip Code',
 'List Price',
 'Original List Price',
 'SqFt Total',
 'Close Price',
 'RATIO Current Price By SQFT',
 'Number Of Stories',
 'Beds Total',
 'Baths Total',
 'Parking Spaces Garage',
 'Lot Size Area',
 'Yr Built',
 'Pool',
 'DOM',
 'Listing Date',
 'Close Date',
 'Expiration Date',
 'Cancelled Date',
 'Status Change Timestamp',
 'List Agent Full Name',
 'Buyers/SubAgent Full Name',
 'HOA',
 'MLS SubArea',
 'Property Sub Type',
 'Property Type',
 'RATIO Close Price By List Price',
 'School District',
 'Subdivision Name']

In [44]:
#rename the unnamed column to Status for transaction status
mls_df = df.rename(columns={"Unnamed: 1": "Status"})
mls_df.dtypes

MLS #                              object
Status                             object
Address                            object
City                               object
Zip Code                           object
List Price                         object
Original List Price                object
SqFt Total                         object
Close Price                        object
RATIO Current Price By SQFT        object
Number Of Stories                  object
Beds Total                         object
Baths Total                        object
Parking Spaces Garage              object
Lot Size Area                      object
Yr Built                           object
Pool                               object
DOM                                object
Listing Date                       object
Close Date                         object
Expiration Date                    object
Cancelled Date                     object
Status Change Timestamp            object
List Agent Full Name              

In [45]:
mlsconvert_df = mls_df.convert_dtypes()

mlsconvert_df.dtypes

MLS #                              string
Status                             string
Address                            string
City                               string
Zip Code                           string
List Price                         string
Original List Price                string
SqFt Total                         string
Close Price                        string
RATIO Current Price By SQFT        string
Number Of Stories                  string
Beds Total                         string
Baths Total                        string
Parking Spaces Garage              string
Lot Size Area                      string
Yr Built                           string
Pool                               string
DOM                                string
Listing Date                       string
Close Date                         string
Expiration Date                    string
Cancelled Date                     string
Status Change Timestamp            string
List Agent Full Name              

In [46]:
# Replace remaining NaN fields across the data set with UnKnown
mls = mlsconvert_df.replace(to_replace = np.nan, value = '')

In [47]:
mls.dtypes

MLS #                              string
Status                             string
Address                            string
City                               string
Zip Code                           string
List Price                         string
Original List Price                string
SqFt Total                         string
Close Price                        string
RATIO Current Price By SQFT        string
Number Of Stories                  string
Beds Total                         string
Baths Total                        string
Parking Spaces Garage              string
Lot Size Area                      string
Yr Built                           string
Pool                               string
DOM                                string
Listing Date                       string
Close Date                         string
Expiration Date                    string
Cancelled Date                     string
Status Change Timestamp            string
List Agent Full Name              

In [48]:
mls.isnull().sum()


MLS #                              0
Status                             0
Address                            0
City                               0
Zip Code                           0
List Price                         0
Original List Price                0
SqFt Total                         0
Close Price                        0
RATIO Current Price By SQFT        0
Number Of Stories                  0
Beds Total                         0
Baths Total                        0
Parking Spaces Garage              0
Lot Size Area                      0
Yr Built                           0
Pool                               0
DOM                                0
Listing Date                       0
Close Date                         0
Expiration Date                    0
Cancelled Date                     0
Status Change Timestamp            0
List Agent Full Name               0
Buyers/SubAgent Full Name          0
HOA                                0
MLS SubArea                        0
P

In [49]:
#clean up the data types

#convert all number data types to float or integer
mls_df[["MLS #"]] = mls_df[["MLS #"]].apply(pd.to_numeric)
mls_df[["List Price"]] = mls_df[["List Price"]].astype(float)
mls_df[["Original List Price"]] = mls_df[["Original List Price"]].astype(float)

mls_df[["SqFt Total", "Close Price", "Number Of Stories", "RATIO Current Price By SQFT", 
        "Beds Total", "Baths Total", "Parking Spaces Garage", "Lot Size Area", "DOM", 
        "RATIO Close Price By List Price", "Zip Code"]] \
= mls_df[["SqFt Total", "Close Price", "Number Of Stories", "RATIO Current Price By SQFT", 
          "Beds Total", "Baths Total", "Parking Spaces Garage", "Lot Size Area", "DOM", 
          "RATIO Close Price By List Price", "Zip Code"]].apply(pd.to_numeric)

mls_df[["Yr Built"]] = mls_df[["Yr Built"]].astype(float)

#convert all dates to date
mls_df[["Listing Date"]] = pd.to_datetime(mls_df["Listing Date"], format='%m/%d/%y')
mls_df[["Close Date"]] = pd.to_datetime(mls_df["Close Date"], format='%m/%d/%y')
mls_df[["Expiration Date"]] = pd.to_datetime(mls_df["Expiration Date"], format='%m/%d/%y')
mls_df[["Cancelled Date"]] = pd.to_datetime(mls_df["Cancelled Date"], format='%m/%d/%y')
mls_df[["Status Change Timestamp"]] = pd.to_datetime(mls_df["Status Change Timestamp"], format='%m/%d/%y')

#convert all strings from objects to string

mls_df[['Status', 'Address', 'City', 'List Agent Full Name', 'Buyers/SubAgent Full Name', 
        'MLS SubArea', 'Property Sub Type', 'Property Type', 'School District', 'Subdivision Name', 'HOA', 'Pool']] \
= mls_df[['Status', 'Address', 'City', 'List Agent Full Name', 'Buyers/SubAgent Full Name', 
        'MLS SubArea', 'Property Sub Type', 'Property Type', 'School District', 'Subdivision Name', 'HOA', 'Pool']].convert_dtypes()
    
mls_df['Status'] = mls_df['Status'].convert_dtypes()

mls_df.dtypes

MLS #                                       int64
Status                                     string
Address                                    string
City                                       string
Zip Code                                    int64
List Price                                float64
Original List Price                       float64
SqFt Total                                float64
Close Price                               float64
RATIO Current Price By SQFT               float64
Number Of Stories                         float64
Beds Total                                  int64
Baths Total                               float64
Parking Spaces Garage                     float64
Lot Size Area                             float64
Yr Built                                  float64
Pool                                       string
DOM                                         int64
Listing Date                       datetime64[ns]
Close Date                         datetime64[ns]


In [50]:
mls_df.dtypes

MLS #                                       int64
Status                                     string
Address                                    string
City                                       string
Zip Code                                    int64
List Price                                float64
Original List Price                       float64
SqFt Total                                float64
Close Price                               float64
RATIO Current Price By SQFT               float64
Number Of Stories                         float64
Beds Total                                  int64
Baths Total                               float64
Parking Spaces Garage                     float64
Lot Size Area                             float64
Yr Built                                  float64
Pool                                       string
DOM                                         int64
Listing Date                       datetime64[ns]
Close Date                         datetime64[ns]


In [51]:
df = pd.DataFrame(mls_df)



In [52]:
mls_csv_data = df.to_csv("CleanedMLSData.csv", index=False, encoding='utf-8')